<div align="right" style="text-align: right"><i>Peter Norvig, Oct 2017<br>pandas Aug 2020<br>Data updated monthly</i></div>

# Bike Stats Code

Code to support the analysis in the notebook [Bike-Stats.ipynb](Bike-Stats.ipynb).

In [6]:
from IPython.core.display import HTML
from typing import Iterator, Iterable, Tuple, List, Dict
from collections import namedtuple
import matplotlib
import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
import re

# Reading Data: `rides` and `yearly`

I saved a bunch of my recorded [Strava](https://www.strava.com/athletes/575579) rides, most of them longer than 25 miles, as [`bikerides.tsv`](bikerides.tsv).  The columns are: the date; the year; a title; the elapsed time of the ride; the length of the ride in miles; and the total climbing in feet, e.g.: 

    Mon, 10/5	2020	Half way around the bay on bay trail	6:26:35	80.05	541
    
I parse the file into the pandas dataframe `rides`, adding derived columns for miles per hour, vertical meters climbed per hour (VAM), grade in feet per mile, grade in percent, and kilometers ridden:

In [7]:
def parse_rides(lines):
    """Parse a bikerides.tsv file."""
    return drop_index(add_columns(pd.read_table(lines, comment='#',
                       converters=dict(hours=parse_hours, feet=parse_int))))

def parse_hours(time: str) -> float: 
    """Parse '4:30:00' => 4.5 hours."""
    hrs = sum(int(x) * 60 ** (i - 2) 
              for i, x in enumerate(reversed(time.split(':'))))
    return round(hrs, 2)

def parse_int(field: str) -> int: return int(field.replace(',', ''))

def add_columns(rides) -> pd.DataFrame:
    """Compute new columns from existing ones."""
    mi, hr, ft = rides['miles'], rides['hours'], rides['feet']
    return rides.assign(
        mph=round(mi / hr, 2),
        vam=round(ft / hr / 3.28084),
        fpm=round(ft / mi),
        pct=round(ft / mi * 100 / 5280, 2),
        kms=round(mi * 1.609, 2),
        km_up=round(ft * 0.0003048, 1))

def drop_index(frame) -> pd.DataFrame:
    """Drop the index column."""
    frame.index = [''] * len(frame)
    return frame

In [65]:
rides  = parse_rides(open('bikerides.tsv'))
yearly = parse_rides(open('bikeyears.tsv')).drop(columns=['date', 'title'])
daily  = yearly.copy()
for name in 'hours miles feet kms km_up'.split():
    daily[name] = round(daily[name].map(lambda x: x / 350), 3 if name == 'km_up' else 1)

# Reading Data: `segments`, `places`, and `tiles`

I picked some representative climbing segments ([`bikesegments.csv`](bikesegments.csv)) with the segment length in miles and climb in feet, along with several of my times on the segment. A line like

    Old La Honda, 2.98, 1255, 28:49, 34:03, 36:44
    
means that this segment of Old La Honda Rd is 2.98 miles long, 1255 feet of climbing, and I've selected three times for my rides on that segment: the fastest, middle, and slowest of the times  that Strava shows. (However, I ended up dropping the slowest time in the charts to make them less busy.)

I keep track of percentage of roads ridden in various places in `'bikeplaceshort.csv'`, which comes from wandrer.earth.

In [41]:
def parse_segments(lines) -> pd.DataFrame:
    """Parse segments into rides. Each ride is a tuple of:
    (segment_title, time,  miles, feet_climb)."""
    records = []
    for segment in lines:
        title, mi, ft, *times = segment.split(',')[:5]
        for time in times:
            records.append((title, parse_hours(time), float(mi), parse_int(ft)))
    return add_columns(pd.DataFrame(records, columns=('title', 'hours', 'miles', 'feet')))

In [42]:
segments = parse_segments(open('bikesegments.csv'))

In [43]:
places = pd.read_table(open('bikeplaceshort.csv'), sep=',', comment='#')

In [50]:
def make_clickable(comment) -> str:
    """Make a clickable link for a pandas dataframe."""
    if '!' not in comment:
        return comment
    anchor, number = comment.split('!')
    return f'<a href="https://www.strava.com/activities/{number}" rel="noopener noreferrer" target="_blank">{anchor}</a>'

def link_date(date) -> str:
    """Make the date into a clickable link."""
    m, d, y = date.split('/')
    return f'<a href="https://www.statshunters.com/?to={y}{m}{d}" rel="noopener noreferrer" target="_blank">{date}</a>'

In [57]:
tiles = drop_index(pd.DataFrame(columns='date square cluster total comment'.split(), data=[
   ('06/30/2023', 13, 689, 2640, 'Rides in east Bay!9298603815'),
   ('04/14/2023', 13, 630, 2595, 'Black Sands Beach connects Marin to max cluster!8891171008'),
   ('03/04/2023', 13, 583, 2574, 'Almaden rides connects Gilroy to max cluster!8654437264'),
   ('10/22/2022', 13, 396, 2495, 'Alviso levees to get to 13x13 max square!8003921626'),
   ('10/16/2022', 12, 393, 2492, 'Milpitas ride connects East Bay to max cluster!7974994605'),
   ('09/08/2022', 11, 300, 2487, 'First started tracking tiles')])
   ).style.format({'comment': make_clickable, 'date': link_date})

# Plotting and Curve-Fitting

In [14]:
plt.rcParams["figure.figsize"] = (12, 6)

def show(X, Y, data, title='', degrees=(2, 3)): 
    """Plot X versus Y and a best fit curve to it, with some bells and whistles."""
    grid(); plt.ylabel(Y); plt.xlabel(X); plt.title(title)
    plt.scatter(X, Y, data=data, c='grey', marker='+')
    X1 = np.linspace(min(data[X]), max(data[X]), 100)
    for degree in degrees:
        F = poly_fit(data[X], data[Y], degree)
        plt.plot(X1, [F(x) for x in X1], '-')
    
def grid(axis='both'): 
    "Turn on the grid."
    plt.minorticks_on() 
    plt.grid(which='major', ls='-', alpha=3/4, axis=axis)
    plt.grid(which='minor', ls=':', alpha=1/2, axis=axis)
    
def poly_fit(X, Y, degree: int) -> callable:
    """The polynomial function that best fits the X,Y vectors."""
    coeffs = np.polyfit(X, Y, degree)[::-1]
    return lambda x: sum(c * x ** i for i, c in enumerate(coeffs)) 

estimator = poly_fit(rides['feet'] / rides['miles'], 
                   rides['miles'] / rides['hours'], 2)

def estimate(miles, feet, estimator=estimator) -> float:
    """Given a ride distance in miles and total climb in feet, estimate time in minutes."""
    return round(60 * miles / estimator(feet / miles))

def top(frame, field, n=20): return frame.sort_values(field, ascending=False).head(n)

# Wandrer Places 

In [15]:
def wandering(places=places, by=['pct']):
    "All those who wander are not lost." # Also try by=['cat', 'pct']
    frame = places.sort_values(by=by, ascending=('pct' not in by))
    M = 1_000_000
    for i, (name, miles, county, pct) in frame.iterrows():
        # Some fiddling to get the format right
        p  = f'{pct:.1f}' if (pct > 0.1) else f'{pct:.3f}'
        mymiles = pct / 100 * miles
        done = f'{rounded(mymiles)}/{rounded(miles)} mi'
        togo = next((f'{rounded(target / 100 * miles - mymiles):>5} mi for {target}%' 
                     for target in (0.02, 0.1, 0.2, 1, 2, 25, 50, 90, 99)
                     if mymiles < target / 100 * miles), '')
        print(f'{county} {p:>5}% {name:25} {done:>15} {togo}') 
        
def rounded(x: float) -> str: 
    """Round x to 3 spaces wide (if possible)."""
    return (rounded(x/1e6) + 'M' if x > 1e6
            else f'{x/1e6:4.2f}M' if x > 1e5
            else f'{round(x):,d}' if x > 10 
            else f'{x:.1f}')

# Pareto Front  

In [16]:
def make_leaders(data):
    """Make a dataframe of leaders in two counties."""
    leaders = pd.DataFrame(data, columns=['Name', 'Initials', 'SMC %', 'SCC %', 'Front?'])
    leaders['SMC miles'] = [round(2814 * d[2] / 100) for d in data]
    leaders['SCC miles'] = [round(7569 * d[3] / 100) for d in data]
    leaders['Total miles'] = leaders['SMC miles'] + leaders['SCC miles']
    leaders['Avg %'] = (leaders['SMC %'] + leaders['SCC %']) / 2
    return drop_index(leaders.sort_values('Avg %', ascending=False))

leaders = make_leaders([ # Data as of Sept 20, 2023 (Name, Initials, SMC, SCC, Frontier?)
    ('Barry Mann', 'BM', 76.97, 30.21, 1),   ('Jason Molenda', 'JM', 7.13, 55.39, 1),  
    ('Peter Norvig', 'PN', 61.56, 32.8, 1), ('Brian Feinberg', 'BF', 32.5, 43.68, 1),
    ('Jim Brooks', 'JB', 4.23, 44.36, 0),    ('Megan Gardner', 'MG', 97.62, 8.69, 1),
    ('Matthew Ring', 'MR', 78.85, 1.48, 0),  ('Elliot  Hoff', 'EF', 52.88, 8.14, 0)])
                   
def pareto_front(leaders):
    ax = leaders.plot('SMC %', 'SCC %', grid=True, kind='scatter')
    front = sorted((x, y) for i, (_, _, x, y, f, *_) in leaders.iterrows() if f)
    ax.plot(*zip(*front), ':'); ax.axis('square'); grid()
    for i, (name, initials, x, y, *_) in leaders.iterrows():
        ax.text(x - 2, y + 2, initials)
    return leaders.drop(columns=['Front?'])

# Eddington Number

In [17]:
def Ed_number(rides, units) -> int:
    """Eddington number: The maximum integer e such that you have bicycled 
    a distance of at least e on at least e days."""
    distances = sorted(rides[units], reverse=True)
    return max(e for e, d in enumerate(distances, 1) if d >= e)

def Ed_gap(distances, target) -> int:
    """The number of rides needed to reach an Eddington number target."""
    return target - sum(distances >= target)

def Ed_gaps(rides, E_km=100, E_mi=67, N=11) -> dict:
    """A table of gaps to Eddington numbers by year."""
    data = [(E_km + d, sum(rides.kms >= E_km + d), Ed_gap(rides.kms,   E_km + d), 
             E_mi + d, sum(rides.miles >= E_mi + d), Ed_gap(rides.miles, E_mi + d))
            for d in range(N)]
    df = pd.DataFrame(data, columns=['kms', 'km rides', 'kms gap', 
                                     'miles', 'miles rides', 'miles gap'])
    return drop_index(df)

def Ed_progress(rides, years=range(2023, 2013, -1)) -> pd.DataFrame:
    """A table of Eddington numbers by year, and a plot."""
    def Ed(year, unit): return Ed_number(rides[rides['year'] <= year], unit)
    data  = [(y, Ed(y, 'kms'), Ed(y, 'miles')) for y in years]
    df = pd.DataFrame(data, columns=['year', 'Ed_km', 'Ed_mi'])
    return drop_index(df)